In [1]:
from TensorHelpers import *
from QFT import *
from MPS import *

import timeit
import numpy as np
from qiskit.visualization import array_to_latex

# Benchmark functionality

In [7]:
def run_benchmark(N, f, fft_start=0.0, fft_end=1.0, max_bond=-1, cutoff=1e-15):
    mps = MPS(N)
    qft = QFT(N)
   
    # Computing benchmark for MPS and MPO
    mps_start = timeit.default_timer() 
    mps = mps.create_MPS([f], fft_start, fft_end, max_bond=max_bond, cutoff=cutoff, reverse=True)
    mps_construct_time = timeit.default_timer() - mps_start
   
    qft_start = timeit.default_timer() 
    qft = qft.create_MPO(max_bond_dim=max_bond, cutoff=cutoff, reverse=True)
    qft_construct_time = timeit.default_timer() - qft_start
    
    qft_apply_start = timeit.default_timer() 
    qft_res = qft.apply(mps)
    qft_apply_time = timeit.default_timer() - qft_apply_start
   
    # Total time 
    mps_construct_apply_time = mps_construct_time + qft_apply_time
    
    print(f"====== Results for benchmark (N={N}, max_bond={max_bond}, cutoff={cutoff}) ======")
    print("MPS Construct Time: ", mps_construct_time)
    print("QFT Construct Time: ", qft_construct_time)
    print("QFT Apply Time: ", qft_apply_time)
    print("MPS + Apply Time: ", mps_construct_apply_time, "\n")
   
    # Comparing accuracy to FFT
    input = np.arange(fft_start, fft_end, step=1/(2**N))
    output = f(input)
    fft_out_vec = np.fft.fft(output).reshape(-1,) / 2**(N/2)
    qft_out_vec = qft_res.to_dense().reshape(-1,)

    print("QFT vs FFT Error: ", np.linalg.norm(fft_out_vec - qft_out_vec))

    return mps_construct_time, mps_construct_apply_time


# Testing Benchmark

In [3]:
run_benchmark(12, lambda x: np.cos(x), max_bond=-1, cutoff=1e-15)

====== Results for benchmark (N=12, max_bond=-1, cutoff=1e-15) ======
MPS Construct Time:  0.0794377149995853
QFT Construct Time:  43.19267000200125
QFT Apply Time:  0.02227201999994577
Total Time:  43.29437973700078 

QFT vs FFT Error:  8.06387047832779e-13


In [4]:
run_benchmark(16, lambda x: np.cos(x), max_bond=-1, cutoff=1e-15)

====== Results for benchmark (N=16, max_bond=-1, cutoff=1e-15) ======
MPS Construct Time:  0.013445404001686256
QFT Construct Time:  1.8696423759993195
QFT Apply Time:  0.04022604800047702
Total Time:  1.9233138280014828 

QFT vs FFT Error:  8.307840334150616e-12


In [5]:
run_benchmark(20, lambda x: np.cos(x), max_bond=-1, cutoff=1e-15)

====== Results for benchmark (N=20, max_bond=-1, cutoff=1e-15) ======
MPS Construct Time:  0.19002685899977223
QFT Construct Time:  2.3637312239989114
QFT Apply Time:  0.014907057000527857
Total Time:  2.5686651399992115 

QFT vs FFT Error:  1.330717896952343e-09


In [6]:
run_benchmark(26, lambda x: np.cos(x), max_bond=-1, cutoff=1e-15)

====== Results for benchmark (N=26, max_bond=-1, cutoff=1e-15) ======
MPS Construct Time:  5.5631886680002935
QFT Construct Time:  3.7497952909998276
QFT Apply Time:  0.019454530998700648
Total Time:  9.332438489998822 

QFT vs FFT Error:  0.00017825112670845865
